In [1]:
import numpy as np
import pandas as pd
import os
import sys


In [2]:
sys.path.insert(0, '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/helpers')
from utils import load_pickle

In [3]:
def bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=5000, upper_percentile=97.5, lower_percentile=2.5, window_size=10.0):
    '''
    To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
    Then for each included subject, sample the chunk to calculate accuracy for this subject
    '''
    
    rng = np.random.RandomState(0)
    
    num_candidate_subjects = len(candidate_subjects)
    assert num_candidate_subjects == 68
    
    bootstrap_accuracy_list = [] # each element is the accuracy of this bootstrap sample (the average accuracy of the selected subjects with their selected chunks in this bootstrap sample)
    for i in range(num_bootstrap_samples):
        print('sample: {}'.format(i))
        #sample the subjects to include for this sample
        subject_location_ix = np.array(range(num_candidate_subjects))
        bootstrap_subject_location_ix = rng.choice(subject_location_ix, num_candidate_subjects, replace=True)
        bootstrap_subjects = candidate_subjects[bootstrap_subject_location_ix]
#         print('subject to include for this sample: {}'.format(bootstrap_subjects))
        
        #for each selected subject, independently resample the chunks to include (as the test set for this subject)
        subject_accuracies = []
        for subject_id in bootstrap_subjects:
            #load the test predictions (for the selected hyper setting) of this subject, and the corresponding true labels
            EnsembleResultSaveDict_this_subject_path = lookup_table.loc[lookup_table['subject_id']==subject_id].experiment_folder.values[0]
            EnsembleResultSaveDict_this_subject = load_pickle(EnsembleResultSaveDict_this_subject_path, 'predictions/result_save_dict.pkl')

            EnsembleTestLogits_this_subject = EnsembleResultSaveDict_this_subject['bestepoch_test_logits']
            TrueLabels_this_subject = EnsembleResultSaveDict_this_subject['bestepoch_test_class_labels']

            #bootstrap the chunks to include for this subject (at this bootstrap sample)
            chunk_location_ix = np.array(range(len(TrueLabels_this_subject)))
            bootstrap_chunk_location_ix = rng.choice(chunk_location_ix, len(TrueLabels_this_subject), replace=True)
            bootstrap_chunks_logits = EnsembleTestLogits_this_subject[bootstrap_chunk_location_ix]
            bootstrap_chunks_true_labels = TrueLabels_this_subject[bootstrap_chunk_location_ix]

            accuracy_this_subject = (bootstrap_chunks_logits.argmax(1) == bootstrap_chunks_true_labels).mean()*100

            subject_accuracies.append(accuracy_this_subject)
        
        
        average_accuracy_this_bootstrap_sample = np.mean(np.array(subject_accuracies))
        bootstrap_accuracy_list.append(average_accuracy_this_bootstrap_sample)
    
    bootstrap_accuracy_array = np.array(bootstrap_accuracy_list)
    
#     #create bootstrap_accuracy_df for input of the bootstrap plotting code by Mike
#     bootstrap_accuracy_df = pd.DataFrame()
#     bootstrap_accuracy_df['sample_id'] = np.arange(num_bootstrap_samples)
#     bootstrap_accuracy_df['accuracy'] = bootstrap_accuracy_array
#     bootstrap_accuracy_df['window_size'] = window_size
    
    accuracy_upper_percentile = np.percentile(bootstrap_accuracy_array, upper_percentile)
    accuracy_lower_percentile = np.percentile(bootstrap_accuracy_array, lower_percentile)

#     return accuracy_upper_percentile, accuracy_lower_percentile, bootstrap_accuracy_df
    return accuracy_upper_percentile, accuracy_lower_percentile




## Logistic Regression

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/FixedTrainValSplit_generic_models/LogisticRegression/binary/'


In [5]:
OtherSubject64_summary_path = '64vs4_summary/'
OtherSubject16_summary_path = '16vs4_summary/'
OtherSubject4_summary_path = '4vs4_summary/'


In [6]:
OtherSubject64_df = pd.read_csv(os.path.join(root_path, OtherSubject64_summary_path, 'AllSubjects_summary.csv'))
OtherSubject16_df = pd.read_csv(os.path.join(root_path, OtherSubject16_summary_path, 'AllSubjects_summary.csv'))
OtherSubject4_df = pd.read_csv(os.path.join(root_path, OtherSubject4_summary_path, 'AllSubjects_summary.csv'))


In [7]:
OtherSubject64_df

,subject_id,bucket,max_validation_accuracy,corresponding_test_accuracy,performance_string,experiment_folder
0,56,TestBucket1,59.644,97.580645,highest validation accuracy: 59.64381720430107...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
1,72,TestBucket1,59.644,25.806452,highest validation accuracy: 59.64381720430107...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
2,86,TestBucket1,59.644,50.537634,highest validation accuracy: 59.64381720430107...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
3,79,TestBucket1,59.644,51.881720,highest validation accuracy: 59.64381720430107...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
4,82,TestBucket2,57.031,72.043011,highest validation accuracy: 57.03125\ncorresp...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
...,...,...,...,...,...,...
63,35,TestBucket16,64.751,69.892473,highest validation accuracy: 64.7513440860215\...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
64,21,TestBucket17,62.836,59.946237,highest validation accuracy: 62.83602150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
65,60,TestBucket17,62.836,82.795699,highest validation accuracy: 62.83602150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
66,29,TestBucket17,62.836,82.526882,highest validation accuracy: 62.83602150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...


### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 31, 32, 34,
 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49,  5, 51, 52,
 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 68, 69,  7, 70, 71,
 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 91, 92, 93,
 94, 95, 97])


In [9]:
len(candidate_subjects)

68

#### OtherSubject64

In [10]:
lookup_table = OtherSubject64_df.copy()
OtherSubject64_upper_percentile, OtherSubject64_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject64:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 692
sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
samp

sample: 1346
sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2615
sample: 2616
sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691

sample: 3251
sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327

sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963

sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599

In [11]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


97.5 percentile is 71.74662001897534; 2.5 percentile is 62.15211891208097


#### OtherSubject16

In [10]:
lookup_table = OtherSubject16_df.copy()
OtherSubject16_upper_percentile, OtherSubject16_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject16:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426
sample: 1427
sample: 1428
sample: 1429

sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061
sample: 2062
sample: 2063
sample: 2064
sample: 2065

sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696
sample: 2697
sample: 2698
sample: 2699
sample: 2700
sample: 2701

sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331
sample: 3332
sample: 3333
sample: 3334
sample: 3335
sample: 3336
sample: 3337

sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966
sample: 3967
sample: 3968
sample: 3969
sample: 3970
sample: 3971

sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602
sample: 4603
sample: 4604
sample: 4605
sample: 4606
sample: 4607

In [11]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


97.5 percentile is 70.45007115749527; 2.5 percentile is 60.86327877925364


#### OtherSubject4

In [12]:
lookup_table = OtherSubject4_df.copy()
OtherSubject4_upper_percentile, OtherSubject4_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject4:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
samp

sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424

sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060

sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696

sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331
sample: 3332

sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966

sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602

In [13]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


97.5 percentile is 64.87992172675521; 2.5 percentile is 54.72792141049969


## Random Forest

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/FixedTrainValSplit_generic_models/RandomForest/binary/'


In [5]:
OtherSubject64_summary_path = '64vs4_summary/'
OtherSubject16_summary_path = '16vs4_summary/'
OtherSubject4_summary_path = '4vs4_summary/'


In [6]:
OtherSubject64_df = pd.read_csv(os.path.join(root_path, OtherSubject64_summary_path, 'AllSubjects_summary.csv'))
OtherSubject16_df = pd.read_csv(os.path.join(root_path, OtherSubject16_summary_path, 'AllSubjects_summary.csv'))
OtherSubject4_df = pd.read_csv(os.path.join(root_path, OtherSubject4_summary_path, 'AllSubjects_summary.csv'))


In [7]:
OtherSubject64_df

,subject_id,bucket,max_validation_accuracy,corresponding_test_accuracy,performance_string,experiment_folder
0,56,TestBucket1,68.775,98.924731,highest validation accuracy: 68.77520161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
1,72,TestBucket1,68.775,42.204301,highest validation accuracy: 68.77520161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
2,86,TestBucket1,68.775,65.591398,highest validation accuracy: 68.77520161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
3,79,TestBucket1,68.775,56.989247,highest validation accuracy: 68.77520161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
4,82,TestBucket2,65.281,75.268817,highest validation accuracy: 65.28057795698925...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
...,...,...,...,...,...,...
63,35,TestBucket16,65.734,69.892473,highest validation accuracy: 65.73420698924731...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
64,21,TestBucket17,68.926,61.827957,highest validation accuracy: 68.92641129032258...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
65,60,TestBucket17,68.926,83.064516,highest validation accuracy: 68.92641129032258...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
66,29,TestBucket17,68.926,72.311828,highest validation accuracy: 68.92641129032258...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...


### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 31, 32, 34,
 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49,  5, 51, 52,
 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 68, 69,  7, 70, 71,
 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 91, 92, 93,
 94, 95, 97])


In [9]:
len(candidate_subjects)

68

#### OtherSubject64

In [10]:
lookup_table = OtherSubject64_df.copy()
OtherSubject64_upper_percentile, OtherSubject64_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject64:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 692
sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
samp

sample: 1346
sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422

sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329

sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965

sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600

In [11]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


97.5 percentile is 76.12270714737508; 2.5 percentile is 68.303190227704


#### OtherSubject16

In [12]:
lookup_table = OtherSubject16_df.copy()
OtherSubject16_upper_percentile, OtherSubject16_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject16:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425

sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061

sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696
sample: 2697

sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331

sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966

sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602

In [13]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


97.5 percentile is 70.78994702719797; 2.5 percentile is 62.96627925363694


#### OtherSubject4

In [14]:
lookup_table = OtherSubject4_df.copy()
OtherSubject4_upper_percentile, OtherSubject4_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject4:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
sample: 778
samp

sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426

sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061

sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695

sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330

sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964

sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598

In [15]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


97.5 percentile is 65.44918168880456; 2.5 percentile is 57.277731657179004


## EEGNet

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/FixedTrainValSplit_generic_models/EEGNet/binary/'


In [5]:
OtherSubject64_summary_path = '64vs4_summary/'
OtherSubject16_summary_path = '16vs4_summary/'
OtherSubject4_summary_path = '4vs4_summary/'


In [6]:
OtherSubject64_df = pd.read_csv(os.path.join(root_path, OtherSubject64_summary_path, 'AllSubjects_summary.csv'))
OtherSubject16_df = pd.read_csv(os.path.join(root_path, OtherSubject16_summary_path, 'AllSubjects_summary.csv'))
OtherSubject4_df = pd.read_csv(os.path.join(root_path, OtherSubject4_summary_path, 'AllSubjects_summary.csv'))


In [7]:
OtherSubject64_df

,subject_id,bucket,max_validation_accuracy,corresponding_test_accuracy,performance_string,experiment_folder
0,56,TestBucket1,65.961,100.000000,highest validation accuracy: 65.96102150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
1,72,TestBucket1,65.961,43.817204,highest validation accuracy: 65.96102150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
2,86,TestBucket1,65.961,57.526882,highest validation accuracy: 65.96102150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
3,79,TestBucket1,65.961,49.193548,highest validation accuracy: 65.96102150537635...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
4,82,TestBucket2,65.171,47.043011,highest validation accuracy: 65.17137096774194...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
...,...,...,...,...,...,...
63,35,TestBucket16,69.254,70.161290,highest validation accuracy: 69.25403225806451...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
64,21,TestBucket17,70.010,50.268817,highest validation accuracy: 70.01008064516128...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
65,60,TestBucket17,70.010,43.279570,highest validation accuracy: 70.01008064516128...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
66,29,TestBucket17,70.010,71.505376,highest validation accuracy: 70.01008064516128...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...


### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 31, 32, 34,
 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49,  5, 51, 52,
 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 68, 69,  7, 70, 71,
 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 91, 92, 93,
 94, 95, 97])


In [9]:
len(candidate_subjects)

68

#### OtherSubject64

In [10]:
lookup_table = OtherSubject64_df.copy()
OtherSubject64_upper_percentile, OtherSubject64_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject64:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
sample: 778
samp

sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426

sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061
sample: 2062

sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696
sample: 2697
sample: 2698

sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331
sample: 3332
sample: 3333

sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966
sample: 3967
sample: 3968

sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602
sample: 4603
sample: 4604

In [11]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


97.5 percentile is 73.84566729917772; 2.5 percentile is 65.52389705882354


#### OtherSubject16

In [12]:
lookup_table = OtherSubject16_df.copy()
OtherSubject16_upper_percentile, OtherSubject16_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject16:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426
sample: 1427
sample: 1428
sample: 1429
sample: 1430

sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061
sample: 2062
sample: 2063
sample: 2064
sample: 2065
sample: 2066

sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696
sample: 2697
sample: 2698
sample: 2699

sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331
sample: 3332
sample: 3333
sample: 3334

sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966
sample: 3967
sample: 3968
sample: 3969

sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602
sample: 4603
sample: 4604
sample: 4605

In [13]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


97.5 percentile is 70.69497153700189; 2.5 percentile is 61.93854759645795


#### OtherSubject4

In [14]:
lookup_table = OtherSubject4_df.copy()
OtherSubject4_upper_percentile, OtherSubject4_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject4:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
samp

sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425

sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3252
sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328

sample: 3885
sample: 3886
sample: 3887
sample: 3888
sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961

sample: 4520
sample: 4521
sample: 4522
sample: 4523
sample: 4524
sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596

In [15]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


97.5 percentile is 59.74897216951295; 2.5 percentile is 50.494149272612276


## DeepConvNet

### For each window size, identify the selected hyperparameter setting for each subject using the AllSubjects_summary.csv


In [4]:
root_path = '/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDataSet2021/experiments/FixedTrainValSplit_generic_models/DeepConvNet/binary/'


In [5]:
OtherSubject64_summary_path = '64vs4_summary/'
OtherSubject16_summary_path = '16vs4_summary/'
OtherSubject4_summary_path = '4vs4_summary/'


In [6]:
OtherSubject64_df = pd.read_csv(os.path.join(root_path, OtherSubject64_summary_path, 'AllSubjects_summary.csv'))
OtherSubject16_df = pd.read_csv(os.path.join(root_path, OtherSubject16_summary_path, 'AllSubjects_summary.csv'))
OtherSubject4_df = pd.read_csv(os.path.join(root_path, OtherSubject4_summary_path, 'AllSubjects_summary.csv'))


In [7]:
OtherSubject64_df

,subject_id,bucket,max_validation_accuracy,corresponding_test_accuracy,performance_string,experiment_folder
0,56,TestBucket1,65.877,96.505376,highest validation accuracy: 65.87701612903226...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
1,72,TestBucket1,65.877,51.612903,highest validation accuracy: 65.87701612903226...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
2,86,TestBucket1,65.877,50.537634,highest validation accuracy: 65.87701612903226...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
3,79,TestBucket1,65.877,48.387097,highest validation accuracy: 65.87701612903226...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
4,82,TestBucket2,66.776,76.612903,highest validation accuracy: 66.77587365591397...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
...,...,...,...,...,...,...
63,35,TestBucket16,69.162,64.784946,highest validation accuracy: 69.16162634408603...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
64,21,TestBucket17,71.900,61.559140,highest validation accuracy: 71.90020161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
65,60,TestBucket17,71.900,66.935484,highest validation accuracy: 71.90020161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...
66,29,TestBucket17,71.900,73.118280,highest validation accuracy: 71.90020161290323...,/cluster/tufts/hugheslab/zhuang12/HCI/NuripsDa...


### Get 97.5 and 2.5 percentile using bootstrapping

To generate 1 bootstrap sample, first sample the 71 subjects to include for this bootstrap sample.
Then for each included subject, sample the chunk to calculate accuracy for this subject

In [8]:
num_bootstrap_samples=5000
upper_percentile=97.5
lower_percentile=2.5

candidate_subjects = np.array([1, 13, 14, 15, 20, 21, 22, 23, 24, 25, 27, 28, 29, 31, 32, 34,
 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49,  5, 51, 52,
 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 68, 69,  7, 70, 71,
 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 91, 92, 93,
 94, 95, 97])


In [9]:
len(candidate_subjects)

68

#### OtherSubject64

In [10]:
lookup_table = OtherSubject64_df.copy()
OtherSubject64_upper_percentile, OtherSubject64_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject64:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
samp

sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423

sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059

sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695

sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331

sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965

sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601

In [11]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject64_upper_percentile, lower_percentile, OtherSubject64_lower_percentile))


97.5 percentile is 74.92113377609108; 2.5 percentile is 66.34230708412397


#### OtherSubject16

In [12]:
lookup_table = OtherSubject16_df.copy()
OtherSubject16_upper_percentile, OtherSubject16_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject16:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
sample: 775
sample: 776
sample: 777
sample: 778
sample: 779
samp

sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422
sample: 1423
sample: 1424
sample: 1425
sample: 1426
sample: 1427

sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057
sample: 2058
sample: 2059
sample: 2060
sample: 2061
sample: 2062
sample: 2063

sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693
sample: 2694
sample: 2695
sample: 2696
sample: 2697
sample: 2698
sample: 2699

sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329
sample: 3330
sample: 3331
sample: 3332
sample: 3333
sample: 3334
sample: 3335

sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965
sample: 3966
sample: 3967
sample: 3968

sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601
sample: 4602
sample: 4603
sample: 4604

In [13]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject16_upper_percentile, lower_percentile, OtherSubject16_lower_percentile))


97.5 percentile is 68.91208096141682; 2.5 percentile is 60.61422754585704


#### OtherSubject4

In [14]:
lookup_table = OtherSubject4_df.copy()
OtherSubject4_upper_percentile, OtherSubject4_lower_percentile = bootstrapping_for_SelectWindowSize(candidate_subjects, lookup_table, num_bootstrap_samples=num_bootstrap_samples, upper_percentile=upper_percentile, lower_percentile=lower_percentile)
print('OtherSubject4:')
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

sample: 692
sample: 693
sample: 694
sample: 695
sample: 696
sample: 697
sample: 698
sample: 699
sample: 700
sample: 701
sample: 702
sample: 703
sample: 704
sample: 705
sample: 706
sample: 707
sample: 708
sample: 709
sample: 710
sample: 711
sample: 712
sample: 713
sample: 714
sample: 715
sample: 716
sample: 717
sample: 718
sample: 719
sample: 720
sample: 721
sample: 722
sample: 723
sample: 724
sample: 725
sample: 726
sample: 727
sample: 728
sample: 729
sample: 730
sample: 731
sample: 732
sample: 733
sample: 734
sample: 735
sample: 736
sample: 737
sample: 738
sample: 739
sample: 740
sample: 741
sample: 742
sample: 743
sample: 744
sample: 745
sample: 746
sample: 747
sample: 748
sample: 749
sample: 750
sample: 751
sample: 752
sample: 753
sample: 754
sample: 755
sample: 756
sample: 757
sample: 758
sample: 759
sample: 760
sample: 761
sample: 762
sample: 763
sample: 764
sample: 765
sample: 766
sample: 767
sample: 768
sample: 769
sample: 770
sample: 771
sample: 772
sample: 773
sample: 774
samp

sample: 1346
sample: 1347
sample: 1348
sample: 1349
sample: 1350
sample: 1351
sample: 1352
sample: 1353
sample: 1354
sample: 1355
sample: 1356
sample: 1357
sample: 1358
sample: 1359
sample: 1360
sample: 1361
sample: 1362
sample: 1363
sample: 1364
sample: 1365
sample: 1366
sample: 1367
sample: 1368
sample: 1369
sample: 1370
sample: 1371
sample: 1372
sample: 1373
sample: 1374
sample: 1375
sample: 1376
sample: 1377
sample: 1378
sample: 1379
sample: 1380
sample: 1381
sample: 1382
sample: 1383
sample: 1384
sample: 1385
sample: 1386
sample: 1387
sample: 1388
sample: 1389
sample: 1390
sample: 1391
sample: 1392
sample: 1393
sample: 1394
sample: 1395
sample: 1396
sample: 1397
sample: 1398
sample: 1399
sample: 1400
sample: 1401
sample: 1402
sample: 1403
sample: 1404
sample: 1405
sample: 1406
sample: 1407
sample: 1408
sample: 1409
sample: 1410
sample: 1411
sample: 1412
sample: 1413
sample: 1414
sample: 1415
sample: 1416
sample: 1417
sample: 1418
sample: 1419
sample: 1420
sample: 1421
sample: 1422

sample: 1981
sample: 1982
sample: 1983
sample: 1984
sample: 1985
sample: 1986
sample: 1987
sample: 1988
sample: 1989
sample: 1990
sample: 1991
sample: 1992
sample: 1993
sample: 1994
sample: 1995
sample: 1996
sample: 1997
sample: 1998
sample: 1999
sample: 2000
sample: 2001
sample: 2002
sample: 2003
sample: 2004
sample: 2005
sample: 2006
sample: 2007
sample: 2008
sample: 2009
sample: 2010
sample: 2011
sample: 2012
sample: 2013
sample: 2014
sample: 2015
sample: 2016
sample: 2017
sample: 2018
sample: 2019
sample: 2020
sample: 2021
sample: 2022
sample: 2023
sample: 2024
sample: 2025
sample: 2026
sample: 2027
sample: 2028
sample: 2029
sample: 2030
sample: 2031
sample: 2032
sample: 2033
sample: 2034
sample: 2035
sample: 2036
sample: 2037
sample: 2038
sample: 2039
sample: 2040
sample: 2041
sample: 2042
sample: 2043
sample: 2044
sample: 2045
sample: 2046
sample: 2047
sample: 2048
sample: 2049
sample: 2050
sample: 2051
sample: 2052
sample: 2053
sample: 2054
sample: 2055
sample: 2056
sample: 2057

sample: 2617
sample: 2618
sample: 2619
sample: 2620
sample: 2621
sample: 2622
sample: 2623
sample: 2624
sample: 2625
sample: 2626
sample: 2627
sample: 2628
sample: 2629
sample: 2630
sample: 2631
sample: 2632
sample: 2633
sample: 2634
sample: 2635
sample: 2636
sample: 2637
sample: 2638
sample: 2639
sample: 2640
sample: 2641
sample: 2642
sample: 2643
sample: 2644
sample: 2645
sample: 2646
sample: 2647
sample: 2648
sample: 2649
sample: 2650
sample: 2651
sample: 2652
sample: 2653
sample: 2654
sample: 2655
sample: 2656
sample: 2657
sample: 2658
sample: 2659
sample: 2660
sample: 2661
sample: 2662
sample: 2663
sample: 2664
sample: 2665
sample: 2666
sample: 2667
sample: 2668
sample: 2669
sample: 2670
sample: 2671
sample: 2672
sample: 2673
sample: 2674
sample: 2675
sample: 2676
sample: 2677
sample: 2678
sample: 2679
sample: 2680
sample: 2681
sample: 2682
sample: 2683
sample: 2684
sample: 2685
sample: 2686
sample: 2687
sample: 2688
sample: 2689
sample: 2690
sample: 2691
sample: 2692
sample: 2693

sample: 3253
sample: 3254
sample: 3255
sample: 3256
sample: 3257
sample: 3258
sample: 3259
sample: 3260
sample: 3261
sample: 3262
sample: 3263
sample: 3264
sample: 3265
sample: 3266
sample: 3267
sample: 3268
sample: 3269
sample: 3270
sample: 3271
sample: 3272
sample: 3273
sample: 3274
sample: 3275
sample: 3276
sample: 3277
sample: 3278
sample: 3279
sample: 3280
sample: 3281
sample: 3282
sample: 3283
sample: 3284
sample: 3285
sample: 3286
sample: 3287
sample: 3288
sample: 3289
sample: 3290
sample: 3291
sample: 3292
sample: 3293
sample: 3294
sample: 3295
sample: 3296
sample: 3297
sample: 3298
sample: 3299
sample: 3300
sample: 3301
sample: 3302
sample: 3303
sample: 3304
sample: 3305
sample: 3306
sample: 3307
sample: 3308
sample: 3309
sample: 3310
sample: 3311
sample: 3312
sample: 3313
sample: 3314
sample: 3315
sample: 3316
sample: 3317
sample: 3318
sample: 3319
sample: 3320
sample: 3321
sample: 3322
sample: 3323
sample: 3324
sample: 3325
sample: 3326
sample: 3327
sample: 3328
sample: 3329

sample: 3889
sample: 3890
sample: 3891
sample: 3892
sample: 3893
sample: 3894
sample: 3895
sample: 3896
sample: 3897
sample: 3898
sample: 3899
sample: 3900
sample: 3901
sample: 3902
sample: 3903
sample: 3904
sample: 3905
sample: 3906
sample: 3907
sample: 3908
sample: 3909
sample: 3910
sample: 3911
sample: 3912
sample: 3913
sample: 3914
sample: 3915
sample: 3916
sample: 3917
sample: 3918
sample: 3919
sample: 3920
sample: 3921
sample: 3922
sample: 3923
sample: 3924
sample: 3925
sample: 3926
sample: 3927
sample: 3928
sample: 3929
sample: 3930
sample: 3931
sample: 3932
sample: 3933
sample: 3934
sample: 3935
sample: 3936
sample: 3937
sample: 3938
sample: 3939
sample: 3940
sample: 3941
sample: 3942
sample: 3943
sample: 3944
sample: 3945
sample: 3946
sample: 3947
sample: 3948
sample: 3949
sample: 3950
sample: 3951
sample: 3952
sample: 3953
sample: 3954
sample: 3955
sample: 3956
sample: 3957
sample: 3958
sample: 3959
sample: 3960
sample: 3961
sample: 3962
sample: 3963
sample: 3964
sample: 3965

sample: 4525
sample: 4526
sample: 4527
sample: 4528
sample: 4529
sample: 4530
sample: 4531
sample: 4532
sample: 4533
sample: 4534
sample: 4535
sample: 4536
sample: 4537
sample: 4538
sample: 4539
sample: 4540
sample: 4541
sample: 4542
sample: 4543
sample: 4544
sample: 4545
sample: 4546
sample: 4547
sample: 4548
sample: 4549
sample: 4550
sample: 4551
sample: 4552
sample: 4553
sample: 4554
sample: 4555
sample: 4556
sample: 4557
sample: 4558
sample: 4559
sample: 4560
sample: 4561
sample: 4562
sample: 4563
sample: 4564
sample: 4565
sample: 4566
sample: 4567
sample: 4568
sample: 4569
sample: 4570
sample: 4571
sample: 4572
sample: 4573
sample: 4574
sample: 4575
sample: 4576
sample: 4577
sample: 4578
sample: 4579
sample: 4580
sample: 4581
sample: 4582
sample: 4583
sample: 4584
sample: 4585
sample: 4586
sample: 4587
sample: 4588
sample: 4589
sample: 4590
sample: 4591
sample: 4592
sample: 4593
sample: 4594
sample: 4595
sample: 4596
sample: 4597
sample: 4598
sample: 4599
sample: 4600
sample: 4601

In [15]:
print('{} percentile is {}; {} percentile is {}'.format(upper_percentile, OtherSubject4_upper_percentile, lower_percentile, OtherSubject4_lower_percentile))


97.5 percentile is 61.74899193548387; 2.5 percentile is 51.59303842504744
